In [1]:
import numpy as np
import pandas as pd
from sklearn import tree #classifier for decision tree
from sklearn.ensemble import RandomForestClassifier #random forest library
#import matplotlib.pyplot as plt
#%pylab inline
#pylab.rcParams['figure.figsize'] = (10,10)


In [2]:
f= open("C:\Users\student12\Desktop\Santander/train_ver2.csv",'r') #loads in the training data set
#the training dataset is too large to load into a dataframe, so we write one month's data into a new CSV below
g = open("C:\Users\student12\Desktop\Santander/decision_tree_training1.csv", 'w')
h = open("C:\Users\student12\Desktop\Santander/test_ver2.csv", 'r')

for line in f:
    date = line[:10]
    if date == '2016-05-28':  #may month used here
        g.write(line)
        
#this is the loop used to filter by date

In [3]:
with open("C:\Users\student12\Desktop\Santander/train_ver2.csv", 'r') as f:
    cols = f.readline().split(',')
with open("C:\Users\student12\Desktop\Santander/test_ver2.csv", 'r') as h:
    cols_t = h.readline().split(',')
    
#everything with _t is the test data set preparation, here were splitting columns by comma, since its a CSV file

In [4]:
cols = [s.replace('"','') for s in cols]
cols_t = [s.replace('"','') for s in cols_t]
#column names seem to be written wierdly, replaced with a normal string format

In [5]:
#loads in the training and test dataset into dataframes
df = pd.read_csv("C:\Users\student12\Desktop\Santander/decision_tree_training1.csv", dtype={'indrel_1mes': str, 'conyuemp': str}, names=cols)
df_t = pd.read_csv("C:\Users\student12\Desktop\Santander/test_ver2.csv", dtype={'indrel_1mes': str, 'conyuemp': str}, names=cols)

C:\Users\student12\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,5,7,8,9,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_t = df_t.ix[1:] #deletes the first row of the test-set since it's the headers duplicated. 

In [7]:
#columns that are mostly empty or useless are deleted here
del df['fecha_dato']
del df['ncodpers']
del df['tipodom']
del df['ult_fec_cli_1t']
del df['conyuemp']
del df_t['fecha_dato']
del df_t['tipodom']
del df_t['ult_fec_cli_1t']
del df_t['conyuemp']

In [8]:
#nifty code, transforms the values in the data-set into a normalised integer format so it can be machine learning processed. 
from sklearn import preprocessing

for f in df.columns:
    if df[f].dtype == 'object':
        print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))

for h in df_t.columns:
    if df_t[h].dtype == 'object':
        print(h)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_t[h].values))
        df_t[h] = lbl.transform(list(df_t[h].values))

ind_empleado
pais_residencia
sexo
fecha_alta
indrel_1mes
tiprel_1mes
indresi
indext
canal_entrada
indfall
nomprov
segmento
ncodpers
ind_empleado
pais_residencia
sexo
age
fecha_alta
ind_nuevo
antiguedad
indrel
indrel_1mes
tiprel_1mes
indresi
indext
canal_entrada
indfall
cod_prov
nomprov
ind_actividad_cliente
renta
segmento


In [9]:
df_t = df_t.iloc[:,:20] #removes product columns from test dataset since they are all Null

In [10]:
df.dtypes #may have to change data type to simpler types

ind_empleado               int64
pais_residencia            int64
sexo                       int64
age                        int64
fecha_alta                 int64
ind_nuevo                float64
antiguedad               float64
indrel                   float64
indrel_1mes                int64
tiprel_1mes                int64
indresi                    int64
indext                     int64
canal_entrada              int64
indfall                    int64
cod_prov                 float64
nomprov                    int64
ind_actividad_cliente    float64
renta                    float64
segmento                   int64
ind_ahor_fin_ult1        float64
ind_aval_fin_ult1        float64
ind_cco_fin_ult1         float64
ind_cder_fin_ult1        float64
ind_cno_fin_ult1         float64
ind_ctju_fin_ult1        float64
ind_ctma_fin_ult1        float64
ind_ctop_fin_ult1        float64
ind_ctpp_fin_ult1        float64
ind_deco_fin_ult1        float64
ind_deme_fin_ult1        float64
ind_dela_f

In [11]:
df_t.dtypes

ncodpers                 int64
ind_empleado             int64
pais_residencia          int64
sexo                     int64
age                      int64
fecha_alta               int64
ind_nuevo                int64
antiguedad               int64
indrel                   int64
indrel_1mes              int64
tiprel_1mes              int64
indresi                  int64
indext                   int64
canal_entrada            int64
indfall                  int64
cod_prov                 int64
nomprov                  int64
ind_actividad_cliente    int64
renta                    int64
segmento                 int64
dtype: object

In [12]:
df =  df.loc[:931435] #the last row of the training dataset acts erratically, easier to remove it.

In [13]:
df.iloc[:,19:] = df.iloc[:,19:].astype('int8') #changes product data types to int8 (1 byte, 8 bits) to save on memory

In [14]:
# a work around for this cell is to fully delete the last row, as it turns out to be very problematic.
#df.loc[931436,'indext'] = 0
#df.loc[931436,'indfall'] = 0 #one value is messing up the boolean aspect of the dataframe

In [15]:
df[['sexo','ind_nuevo','indext','indfall','ind_actividad_cliente','indresi']] = df[['sexo','ind_nuevo','indext','indfall','ind_actividad_cliente','indresi']].astype('bool')

In [16]:
for f in df.columns:
    if df[f].dtype == 'float64':
        print(f)
        df[f] = df[f].astype('float16') #changes float64 datatypes to float16 to conserve memory (decision tree algorithm doesnt like data-types too big) 

antiguedad
indrel
cod_prov
renta


In [17]:
for f in df.columns:
    if df[f].dtype == 'int64':
        print(f)
        df[f] = df[f].astype('int16')
        
for h in df_t.columns[1:]:
    if df_t[h].dtype == 'int64':
        print(h)
        df_t[h] = df_t[h].astype('int8')

ind_empleado
pais_residencia
age
fecha_alta
indrel_1mes
tiprel_1mes
canal_entrada
nomprov
segmento
ind_empleado
pais_residencia
sexo
age
fecha_alta
ind_nuevo
antiguedad
indrel
indrel_1mes
tiprel_1mes
indresi
indext
canal_entrada
indfall
cod_prov
nomprov
ind_actividad_cliente
renta
segmento


In [18]:
df_t.dtypes

ncodpers                 int64
ind_empleado              int8
pais_residencia           int8
sexo                      int8
age                       int8
fecha_alta                int8
ind_nuevo                 int8
antiguedad                int8
indrel                    int8
indrel_1mes               int8
tiprel_1mes               int8
indresi                   int8
indext                    int8
canal_entrada             int8
indfall                   int8
cod_prov                  int8
nomprov                   int8
ind_actividad_cliente     int8
renta                     int8
segmento                  int8
dtype: object

In [19]:
#back up in case of everything dying
df_backup = df  
df_t_backup = df_t

In [20]:
#sets the dataframe index to the customer code, useful later when suggesting products to specific customers
df_t.set_index(['ncodpers'],inplace=True)

In [21]:
df_t.head()

,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
ncodpers,,,,,,,,,,,,,,,,,,,
0,2,36,1,52,0,0,33,0,0,0,1,0,24,0,49,30,1,21,0
138992,3,36,0,32,-91,0,32,0,0,1,1,0,24,0,2,2,0,0,1
138993,3,36,1,18,-91,0,32,0,0,0,1,0,-106,0,21,18,1,0,2
138994,3,36,0,18,-91,0,32,0,0,1,1,0,-106,0,7,8,0,-82,2
138995,3,36,0,18,-91,0,32,0,0,1,1,0,-106,0,6,7,0,-15,2


In [22]:
# A list of products and features, will be used when defining targets and features for the ML algorithm.
products = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1\n']
descriptors = ['ind_actividad_cliente','segmento','tiprel_1mes','fecha_alta','canal_entrada','age','sexo']

In [23]:
target = df[products].values
features = df[descriptors].values
#target_t = df_t[products].values
features_t = df_t[descriptors].values

In [24]:
#this is where the training of the model actually occurs
#my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = RandomForestClassifier()
my_tree_one = my_tree_one.fit(features,target)

In [25]:
#returns a percentage score of how accurate the predictions were in relation to the actual results, is a viable method since this is a supervised learning model
my_tree_one.score(features,target)

0.82633374703146545

In [26]:
#this is where the test set is fed in for predictions
predictions = my_tree_one.predict(features_t)

In [27]:
#trans = np.transpose(predictions)

In [28]:
predictDF = pd.DataFrame(predictions)


In [29]:
predictDF = predictDF.astype('int32') #make dataframe smaller for memory reasons


In [30]:
predictDF.columns = products #renaming column names to the product names 

In [31]:
predictDF.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
#sets the index as the customer code, so the predictions can be matched to a customer
predictDF.set_index(df_t.index,inplace=True)

In [33]:
#real wizardy occurs here, pivots the columns into an index, this is done so the values can be filtered in the next step
wow = pd.DataFrame(predictDF.unstack())

In [34]:
filtered = wow[wow[0] == 1] #filters the data frame to only return the products predicted as a yes, no need for the others
filtered = filtered.ix[:,1:] #returns the product:customer to be recommended to

In [35]:
#filtered.to_csv("C:\GIT\Santander/Ali/2nd-submission.csv")

In [36]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.2.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [44]:
import xgboost as xgb

In [56]:
dfeatures = xgb.DMatrix(features)
dtarget = xgb.DMatrix(target)
dtrain = xgb.DMatrix(features, label=target)


TypeError: only length-1 arrays can be converted to Python scalars

In [55]:
# XGBOOST
model = xgboost.XGBClassifier()
model.fit(dfeatures, dtarget)


ValueError: bad input shape ()

In [ ]:
# make predictions for test data
y_pred = model.predict(features_t)
predictions = [round(value) for value in y_pred]

In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


In [54]:
dfeatures

TypeError: 'NoneType' object is not callable